# Numeric SAM Demo Notebook

In [2]:
import os
import json
from pathlib import Path

from pddl_plus_parser.lisp_parsers import DomainParser, ProblemParser, TrajectoryParser

from sam_learning.numeric_sam import NumericSAMLearner

os.environ["CONVEX_HULL_ERROR_PATH"] = "tests/temp_files/ch_error.txt"

In [3]:
EXAMPLES_PATH = Path("demos_data")

depot_partial_domain = DomainParser(EXAMPLES_PATH / "depot_numeric.pddl", partial_parsing=True).parse_domain()
depot_problem = ProblemParser(EXAMPLES_PATH / "pfile2.pddl", domain=depot_partial_domain).parse_problem()
depot_observation = TrajectoryParser(depot_partial_domain, depot_problem).parse_trajectory(EXAMPLES_PATH / "test_numeric_trajectory")

In [4]:
with open(EXAMPLES_PATH / "depot_fluents_map.json", "rt") as json_file:
    depot_fluents_map = json.load(json_file)


In [5]:
numeric_sam = NumericSAMLearner(depot_partial_domain, depot_fluents_map)

print(numeric_sam.partial_domain.to_pddl())

(define (domain depot)
(:requirements :typing :fluents :disjunctive-preconditions :negative-preconditions :equality :universal-preconditions)
(:types 	place locatable - object
	depot distributor - place
	truck hoist surface - locatable
	pallet crate - surface
)

(:predicates (at ?x - locatable ?y - place)
	(on ?x - crate ?y - surface)
	(in ?x - crate ?y - truck)
	(lifting ?x - hoist ?y - crate)
	(available ?x - hoist)
	(clear ?x - surface)
)

(:functions (load_limit ?t - truck)
	(current_load ?t - truck)
	(weight ?c - crate)
	(fuel-cost )
)

(:action drive
	:parameters (?x - truck ?y - place ?z - place)
	:precondition (and )
	:effect (and  
		))

(:action lift
	:parameters (?x - hoist ?y - crate ?z - surface ?p - place)
	:precondition (and )
	:effect (and  
		))

(:action drop
	:parameters (?x - hoist ?y - crate ?z - surface ?p - place)
	:precondition (and )
	:effect (and  
		))

(:action load
	:parameters (?x - hoist ?y - crate ?z - truck ?p - place)
	:precondition (and )
	:effect (an

In [6]:
learned_model, learning_metadata = numeric_sam.learn_action_model([depot_observation])
print(learning_metadata)

There are too few independent rows of data! cannot solve linear equations for action - load!
There are too few independent rows of data! cannot solve linear equations for action - unload!


{'drive': 'OK', 'lift': 'OK', 'drop': 'OK', 'load': 'OK', 'unload': 'OK', 'learning_time': '0.24134302139282227'}


In [7]:
print(learned_model.to_pddl())

(define (domain depot)
(:requirements :typing :fluents :disjunctive-preconditions :negative-preconditions :equality :universal-preconditions)
(:types 	place locatable - object
	depot distributor - place
	truck hoist surface - locatable
	pallet crate - surface
)

(:predicates (at ?x - locatable ?y - place)
	(on ?x - crate ?y - surface)
	(in ?x - crate ?y - truck)
	(lifting ?x - hoist ?y - crate)
	(available ?x - hoist)
	(clear ?x - surface)
)

(:functions (load_limit ?t - truck)
	(current_load ?t - truck)
	(weight ?c - crate)
	(fuel-cost )
)

(:action drive
	:parameters (?x - truck ?y - place ?z - place)
	:precondition (and (at ?x ?y)
	(not (at ?x ?z))(not (= ?y ?z)))
	:effect (and (not (at ?x ?y))
		(at ?x ?z)
(increase (fuel-cost ) 10.0)))

(:action lift
	:parameters (?x - hoist ?y - crate ?z - surface ?p - place)
	:precondition (and (at ?z ?p)
	(not (clear ?z))
	(clear ?y)
	(available ?x)
	(at ?x ?p)
	(at ?y ?p)
	(not (lifting ?x ?y))
	(on ?y ?z))
	:effect (and (not (available ?x))
	